# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [967]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [968]:
df = pd.read_csv("data/AviationData.csv", encoding="ISO-8859-1")

df.duplicated().sum() #check for duplicates. Therea are none

df.head() #see if NaNs are in by looking at first few rows (there are a lot)

df.isnull().sum() #check for NaN by column
nan_percentages = df.isnull().mean() * 100 #will show me the nan


df.describe() #summary statistics

print(df.columns)


/var/folders/hq/0d2ft2dx2xx1hw4vlltc8v7h0000gn/T/ipykernel_6267/1668717694.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/AviationData.csv", encoding="ISO-8859-1")


Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [969]:
#filer columns with a lof NaNs and/or columns that are irrelevant to client's question
display_columns = [
    "Event.Date", "Injury.Severity", "Aircraft.damage",
    "Make", "Model", "Amateur.Built",
    "Number.of.Engines", "Engine.Type", "Total.Fatal.Injuries",
    "Total.Serious.Injuries", "Total.Minor.Injuries", "Total.Uninjured",
    "Weather.Condition", "Broad.phase.of.flight", "Purpose.of.flight"
]

df = df.filter(items=display_columns) #filter the columns I do not need
df = df[df["Amateur.Built"] != "Yes"] #boolean masking. Client is only asking about professionally made aircrafts

df["Event.Date"]= pd.to_datetime(df["Event.Date"]) #change to datetime format

earliest_date = pd.Timestamp("1983-01-01") #filter by earliest date being 1983
df = df[df["Event.Date"] >= earliest_date]

df = df[df["Number.of.Engines"] > 0] #more than 0 engines to be an aircraft. Same idea as setting earliest date. I am guessing maybe it would be something like a hot air balloon

df.head()

,Event.Date,Injury.Severity,Aircraft.damage,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Purpose.of.flight
3601,1983-01-01,Non-Fatal,Substantial,Cessna,182P,No,1.0,Reciprocating,0.0,0.0,1.0,3.0,VMC,Approach,Personal
3602,1983-01-01,Non-Fatal,Substantial,Cessna,182RG,No,1.0,Reciprocating,0.0,0.0,0.0,2.0,VMC,Landing,Personal
3603,1983-01-01,Non-Fatal,Substantial,Cessna,182P,No,1.0,Reciprocating,0.0,0.0,0.0,1.0,VMC,Takeoff,Personal
3604,1983-01-01,Non-Fatal,Substantial,Piper,PA-28R-200,No,1.0,Reciprocating,0.0,0.0,2.0,0.0,VMC,Approach,Personal
3605,1983-01-01,Non-Fatal,Substantial,Cessna,140,No,1.0,Reciprocating,0.0,0.0,0.0,2.0,VMC,Landing,Instructional


### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [970]:
#I am going to use a percentage as my metric

injury_columns = ["Total.Fatal.Injuries", "Total.Serious.Injuries", "Total.Minor.Injuries", "Total.Uninjured"] #all injury columns for ease
df[injury_columns] = df[injury_columns].fillna(0) #set any NaN to 0. I think data would have been reported had there been injuries or fatalities 
#create total occupants variable
df["Total.Occupants"] = (df["Total.Fatal.Injuries"] + df["Total.Serious.Injuries"] + df['Total.Minor.Injuries'] + df['Total.Uninjured'])

df = df[df["Total.Occupants"] > 0] #only keep accidents where there was at least 1 person on board (ie. an actual accident)

# Combine fatal and serious injuries
df["Fatal.Serious.Injuries"] = df["Total.Fatal.Injuries"] + df["Total.Serious.Injuries"]

#here is my percentage metric
df["Fatal.Serious.Injury.Percent"] = (df["Fatal.Serious.Injuries"] / df["Total.Occupants"]) * 100

df.head(10)

,Event.Date,Injury.Severity,Aircraft.damage,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Purpose.of.flight,Total.Occupants,Fatal.Serious.Injuries,Fatal.Serious.Injury.Percent
3601,1983-01-01,Non-Fatal,Substantial,Cessna,182P,No,1.0,Reciprocating,0.0,0.0,1.0,3.0,VMC,Approach,Personal,4.0,0.0,0.0
3602,1983-01-01,Non-Fatal,Substantial,Cessna,182RG,No,1.0,Reciprocating,0.0,0.0,0.0,2.0,VMC,Landing,Personal,2.0,0.0,0.0
3603,1983-01-01,Non-Fatal,Substantial,Cessna,182P,No,1.0,Reciprocating,0.0,0.0,0.0,1.0,VMC,Takeoff,Personal,1.0,0.0,0.0
3604,1983-01-01,Non-Fatal,Substantial,Piper,PA-28R-200,No,1.0,Reciprocating,0.0,0.0,2.0,0.0,VMC,Approach,Personal,2.0,0.0,0.0
3605,1983-01-01,Non-Fatal,Substantial,Cessna,140,No,1.0,Reciprocating,0.0,0.0,0.0,2.0,VMC,Landing,Instructional,2.0,0.0,0.0
3607,1983-01-02,Non-Fatal,Destroyed,Cessna,340A,No,2.0,Reciprocating,0.0,0.0,4.0,0.0,VMC,Takeoff,Personal,4.0,0.0,0.0
3608,1983-01-02,Fatal(2),Destroyed,North American,T-6G,No,1.0,Reciprocating,2.0,0.0,0.0,0.0,VMC,Maneuvering,Personal,2.0,2.0,100.0
3609,1983-01-02,Fatal(3),Destroyed,Piper,PA-24-250,No,1.0,Reciprocating,3.0,0.0,0.0,0.0,IMC,Cruise,Personal,3.0,3.0,100.0
3610,1983-01-02,Non-Fatal,Substantial,Piper,PA-32-301R,No,1.0,Reciprocating,0.0,0.0,0.0,2.0,VMC,Maneuvering,Personal,2.0,0.0,0.0
3611,1983-01-02,Non-Fatal,Substantial,Beech,V-35B,No,1.0,Reciprocating,0.0,0.0,0.0,3.0,VMC,Takeoff,Personal,3.0,0.0,0.0


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [971]:
df["Aircraft.damage"].unique() #see different variables should be Substantial, Destroyed, Minor, Unknown

df["Aircraft.damage"] = df["Aircraft.damage"].fillna("Unknown") #change out NaNs to "Unknown"

def is_destroyed(damage): #creating a function to make destroyed or not data
    if damage == 'Destroyed':
        return 1
    else:
        return 0

df["Aircraft.Destroyed"] = df["Aircraft.damage"].apply(is_destroyed)

df.head()



,Event.Date,Injury.Severity,Aircraft.damage,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Purpose.of.flight,Total.Occupants,Fatal.Serious.Injuries,Fatal.Serious.Injury.Percent,Aircraft.Destroyed
3601,1983-01-01,Non-Fatal,Substantial,Cessna,182P,No,1.0,Reciprocating,0.0,0.0,1.0,3.0,VMC,Approach,Personal,4.0,0.0,0.0,0
3602,1983-01-01,Non-Fatal,Substantial,Cessna,182RG,No,1.0,Reciprocating,0.0,0.0,0.0,2.0,VMC,Landing,Personal,2.0,0.0,0.0,0
3603,1983-01-01,Non-Fatal,Substantial,Cessna,182P,No,1.0,Reciprocating,0.0,0.0,0.0,1.0,VMC,Takeoff,Personal,1.0,0.0,0.0,0
3604,1983-01-01,Non-Fatal,Substantial,Piper,PA-28R-200,No,1.0,Reciprocating,0.0,0.0,2.0,0.0,VMC,Approach,Personal,2.0,0.0,0.0,0
3605,1983-01-01,Non-Fatal,Substantial,Cessna,140,No,1.0,Reciprocating,0.0,0.0,0.0,2.0,VMC,Landing,Instructional,2.0,0.0,0.0,0


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [972]:
# df = df.dropna(subset=['Make']) #drop any NaNs
df["Make"] = df["Make"].str.strip().str.title() #set standard font

number_makes= df["Make"].value_counts() 
number_makes #shows series of makes

threshold = 50 #how many we are allowing minimally

#see makes that meet or exceed threshold
makes_keep = number_makes[number_makes >= threshold].index
df = df[df["Make"].isin(makes_keep)]

merge_names = {"Air Tractor Inc": "Air Tractor", "Airbus Industrie": "Airbus", "Aviat Aircraft Inc": "Aviat", "Bombardier Inc": "Bombardier",
               "Cirrus Design Corp": "Cirrus", "Cirrus Design Corp.": "Cirrus",
               "Dehavilland": "De Havilland", "Ercoupe (Eng & Research Corp.)": "Ercoupe", "Grumman Acft Eng Cor-Schweizer": "Grumman", "Grumman American": "Grumman",
                "Grumman-Schweizer": "Grumman", "Robinson Helicopter": "Robinson", "Robinson Helicopter Company": "Robinson", "Rockwell International": "Rockwell"}

#chose not to merge Boeing and Boeing Stearman (Boeing Stearman is a subsidiary). Same with American and American Champion
df["Make"].value_counts() #see how many are left (77)

df["Make"] = df["Make"].map(merge_names).fillna(df["Make"])

df["Make"].value_counts()

Make
Cessna                        24741
Piper                         13585
Beech                          4811
Bell                           2281
Boeing                         1572
                              ...  
Bombardier                       54
American Champion Aircraft       53
Smith, Ted Aerostar              51
Boeing Stearman                  50
Agusta                           50
Name: count, Length: 64, dtype: int64

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [973]:
df = df.dropna(subset=["Model"]) #drop any NaNs

df["Model"].value_counts().head(50) #show me a good sample size

df["Make.Model"] = df["Make"] + " " + df["Model"] #create new column

make_models = df["Make.Model"].value_counts().nunique()#Each make has AT LEAST 1 model associated with it. Client wants to know which SPECIFIC aircrafts are the safest

models_by_make = df.groupby('Make')['Model'].unique()

# print(models_by_make)
# def check_unique_models(df, make_col = "Make", model_col = "Model"):
#     assert len(models) == len(set(models))

# check_unique_models(df, "Make", "Model")
model_counts = df.groupby(['Make', 'Model']).size()

#I believe that there is 



### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [974]:
#Note I am choosing to deal with the NaNs here
df.columns = df.columns.str.strip().str.title() #change font *after removal... gave me errors

#Engine Type
df["Engine.Type"].unique() #see uniques
df["Engine.Type"] = df["Engine.Type"].replace({"None": "Unknown", "UNK": "Unknown"}) #change uniques to unkown

#Weather Conditions
df["Weather.Condition"].unique() #same as above
df["Weather.Condition"] = df["Weather.Condition"].replace({"UNK": "Unknown", "Unk": "Unknown"}) #change uniques to unkown
#Number of Engines
df["Number.Of.Engines"].unique() #looks okay to me just floats
# print(df['Number.Of.Engines'].head(10))
#Purpose of Flight
df["Purpose.Of.Flight"].unique() #to see uniques.

#Broad Phase of Flight
df["Broad.Phase.Of.Flight"].unique()
df["Broad.Phase.Of.Flight"] = df["Broad.Phase.Of.Flight"].replace({"Other": "Unknown"})




### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [975]:
nan_counts = df.isnull().sum()
print(nan_counts)
#these were "dropped" when I did the keep columns above
df["Amateur.Built"] = df["Amateur.Built"].fillna("No") #count was 12. based on sample size I am filling in with no
df["Weather.Condition"] = df["Weather.Condition"].fillna("Unknown") # count was 1036 filled in Nans with unknowns 
df["Purpose.Of.Flight"] = df["Purpose.Of.Flight"].fillna("Unknown") #count was 2546. 
df["Engine.Type"] = df["Engine.Type"].fillna("Unknown") #count was 1697. Note that all of these values are low % so I felt comfortable doing this
#not sure I am to remove Broad Phase of Flight, but will not use for analysis.

df.head()




Event.Date                          0
Injury.Severity                     0
Aircraft.Damage                     0
Make                                0
Model                               0
Amateur.Built                      12
Number.Of.Engines                   0
Engine.Type                      1697
Total.Fatal.Injuries                0
Total.Serious.Injuries              0
Total.Minor.Injuries                0
Total.Uninjured                     0
Weather.Condition                1036
Broad.Phase.Of.Flight           15508
Purpose.Of.Flight                2546
Total.Occupants                     0
Fatal.Serious.Injuries              0
Fatal.Serious.Injury.Percent        0
Aircraft.Destroyed                  0
Make.Model                          0
dtype: int64


,Event.Date,Injury.Severity,Aircraft.Damage,Make,Model,Amateur.Built,Number.Of.Engines,Engine.Type,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.Of.Flight,Purpose.Of.Flight,Total.Occupants,Fatal.Serious.Injuries,Fatal.Serious.Injury.Percent,Aircraft.Destroyed,Make.Model
3601,1983-01-01,Non-Fatal,Substantial,Cessna,182P,No,1.0,Reciprocating,0.0,0.0,1.0,3.0,VMC,Approach,Personal,4.0,0.0,0.0,0,Cessna 182P
3602,1983-01-01,Non-Fatal,Substantial,Cessna,182RG,No,1.0,Reciprocating,0.0,0.0,0.0,2.0,VMC,Landing,Personal,2.0,0.0,0.0,0,Cessna 182RG
3603,1983-01-01,Non-Fatal,Substantial,Cessna,182P,No,1.0,Reciprocating,0.0,0.0,0.0,1.0,VMC,Takeoff,Personal,1.0,0.0,0.0,0,Cessna 182P
3604,1983-01-01,Non-Fatal,Substantial,Piper,PA-28R-200,No,1.0,Reciprocating,0.0,0.0,2.0,0.0,VMC,Approach,Personal,2.0,0.0,0.0,0,Piper PA-28R-200
3605,1983-01-01,Non-Fatal,Substantial,Cessna,140,No,1.0,Reciprocating,0.0,0.0,0.0,2.0,VMC,Landing,Instructional,2.0,0.0,0.0,0,Cessna 140


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [976]:
df.head(10)
df.to_csv("Cleaned_Craft_Data.csv", index =False) #had to go back and figure this out (index= False)...keep note!
